<a href="https://colab.research.google.com/github/okadatsuneo/Plateau_import/blob/main/op_osm%E5%87%A6%E7%90%86_typebuilding%E5%89%8A%E9%99%A4_%E3%83%95%E3%82%A9%E3%83%AB%E3%83%80%E5%86%85%E5%85%A8%E9%83%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Plateauインポートの_op.osmファイルの処理：type=buildingのリレーションを解除する。

入力ファイル:＊＊＊_op.osm、出力ファイル:＊＊＊.op_2.osm

特定のフォルダ内の複数ファイルをまとめて処理する。

In [ ]:
import os
import xml.etree.ElementTree as ET
from pathlib import Path

Googleドライブにあるファイルを読み込む。

In [ ]:
#ドライブのマウントはセッションが切れると解除される。ファイルも削除されるので、MyDrive下に置く。
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


_op.osm のOSMファイルの読み込みをして処理をする。_op2.osmとして出力する。

In [ ]:
# 処理対象のフォルダを指定
folder_path = Path('/content/drive/MyDrive/python/tmp/')

def process_folder(folder_path):
    # フォルダ内のすべてのXMLファイルに対して処理
    for input_file in folder_path.glob('*.osm'):
        process_xml(input_file)

def process_xml(input_file):
    # 出力ファイル名の作成
    base_name, extension = os.path.splitext(input_file)
    output_file = f'{base_name}_2{extension}'

    # XMLファイルをパースする
    tree = ET.parse(input_file)
    # ルート要素を取得
    root = tree.getroot()

    # リストを作成して削除する要素を追加
    elements_to_remove = []

    # k="type" and v="multipolygon" を持つ "relation" に対しての処理
    for relation in root.findall(".//relation/tag[@k='type'][@v='multipolygon']/.."):
        # 'k="building:part"' を 'k="building"' に書き換える
            building_part_tag = relation.find(".//tag[@k='building:part']")
            if building_part_tag is not None:
                building_part_tag.set('k', 'building')

    # k="type" and v="building" を持つ "relation" に対しての処理
    # "/.."をつけることで、tag の条件を満たす要素を選択するのではなく、その親要素を選択するようにしています。
    for relation in root.findall(".//relation/tag[@k='type'][@v='building']/.."):
        # 'role'="part" かつ 'ref' を持つ "way" を取得:OK
        part_ways = relation.findall(".//member[@role='part'][@type='way']")
        for part_way in part_ways:
            ref_value = part_way.get('ref')
            #print(ref_value)
            way = root.find(f".//way[@id='{ref_value}']")
            if way is not None:
                #print(f"Modifying way with id={way.get('id')}: {ET.tostring(way)}")
                # 'k="building:part"' を 'k="building"' に書き換える
                building_part_tag = way.find(".//tag[@k='building:part']")
                if building_part_tag is not None:
                    building_part_tag.set('k', 'building')
        # 'role'="outline" かつ 'ref' を持つ "way" を取得
        part_ways = relation.findall(".//member[@role='outline'][@type='way']")
        for part_way in part_ways:
            ref_value = part_way.get('ref')
            way = root.find(f".//way[@id='{ref_value}']")
            if way is not None:
                #print(f"Removing way with id={way.get('id')}: {ET.tostring(way)}")
                elements_to_remove.append(way)

        # "relation" タグが存在する場合にはそれを削除
        #print(f"Removing relation with id={relation.get('id')}: {ET.tostring(relation)}")
        elements_to_remove.append(relation)

    # リストにある要素を削除
    for element in elements_to_remove:
        if element in root:
            root.remove(element)

    # 出力時に encoding='utf-8' を指定してファイルに書き込む
    tree.write(output_file, encoding='utf-8')

# フォルダ内のすべてのXMLファイルに対して処理を行う
process_folder(folder_path)

print("処理が完了しました。")

処理が完了しました。
